
### Section 1.1 – Dataset Clipping and Folder Management Based on HUC Boundaries

The following script automates raster clipping and folder organization based on HUC boundaries. It is designed to streamline the setup of input datasets for the flood risk modeling workflow.

---

#### 📁 Required Datasets

We recommend preparing all datasets listed in FAU (2020), including:
- Digital Elevation Model (DEM)
- Water Table Elevation
- Water Holding Capacity
- Impervious Mask
- Water Mask
- Design Rainfall Rasters

To maintain consistency and ensure smooth processing, keep variable names consistent with the naming convention used in the script. If you use custom names, you may need to manually modify the replacement logic in the script.

---

#### ✅ 1. Set the Project Directory

Update the `project_dir` variable to the root folder that contains your geodatabase, raster datasets, and where output subdirectories will be created.

```python
project_dir = r"X:\path\to\your\project"
```

---

#### 🗂 2. Configure HUC Boundary Settings

In the script, configure the following parameters based on your geodatabase:

```python
gdb_name = "Homestead"         # Name of your geodatabase (without '.gdb')
huc_fname = "HUC_Boundaries"   # Name of the HUC boundary feature class
huc_col = "HUC_NAME"           # Column name for HUC boundary names
id_col = "HUC_ID"              # Column name for unique HUC identifiers
```

---

#### 📌 3. Define Dataset Paths

The script uses a `datasets` dictionary to locate and organize raster inputs. Update it with the paths to your datasets:

```python
datasets = {
    "dem_ft": r"path\to\DEM.tif",
    "wte_ft": r"path\to\WaterTable.tif",
    "whc_ratio": r"path\to\WHC.tif",
    ...
}
```

Use descriptive keys that match the naming convention to simplify downstream processing.

---

#### ▶️ 4. Run the Script

Before running the script:

- Ensure ArcGIS Pro and the Python environment are properly installed.
- Confirm you have read/write permissions in the specified directories.

When executed:
- If your HUC feature class contains **multiple rows**, the script will:
  - Create a subdirectory under `HUC12s/` for each HUC.
  - Clip each raster to its respective HUC extent.
- If the HUC feature class contains **only one row**, clipped datasets will be saved to the `basedir/` folder.

---

#### 🗃 5. Mosaic Creation

After clipping is completed:
- The script groups the resulting raster files by name and extension.
- Mosaics are generated for each group and saved in the `mosaic/` directory under `project_dir`.

Ensure the `mosaic/` directory exists, or the script is set to create it before saving.

---

#### ✅ Final Notes

- Maintain consistent naming for all rasters.
- Check spatial reference and alignment across input datasets.
- This script is an essential first step in preparing your data for flood scenario modeling and risk assessment.


In [1]:
%%time
import os
import glob
from arcpy.sa import *
arcpy.env.addOutputsToMap = False
# Enable overwriting of output files
arcpy.env.overwriteOutput = True
# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")
# Set the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'

# Settings for the HUC boundary
gdb_name = "Homestead"
huc_fname = "HUC12s_Boundary"  # Name of the feature class
huc_col = "NAME"  # Column representing the name of the HUCs
id_col = "HUC12"  # Column representing the ID of the HUCs

# Replace the paths of the datasets (please check the data management guide for details)
datasets = {
    'dem_ft': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\dem_ft_mosaic.tif',
    'wte_ft': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\wte_ft.tif',
    'whc_ratio': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\whc.tif',
    'impervious': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\impervious.tif',
    'water': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\water.tif',
    'rain_1d5yr_in': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\rain_5y1d_in.tif',
    'rain_1d10yr_in': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\rain_10y1d_in.tif',
    'rain_1d100yr_in': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\rain_25y3d_in.tif',
    'rain_3d25yr_in': r'X:\amandal2023$\Working\Homestead\Homestead_City\Homestead_City\rain_100y1d_in.tif'
}

# Enable overwriting of output files
arcpy.env.overwriteOutput = True

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

# Set workspace to the project's directory
arcpy.env.workspace = project_dir

# Path to the geodatabase
gdb_path = os.path.join(project_dir, f"{gdb_name}.gdb")
huc_path = os.path.join(gdb_path, huc_fname)

# Function to create directories if they don't exist
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

# Function to clip and save rasters
def clip_and_save(huc, output_dir):
    saved_paths = []
    for key, raster in datasets.items():
        output_raster = os.path.join(output_dir, f"{key}.tif")
        try:
            # Perform the extraction
            arcpy.sa.ExtractByMask(raster, huc).save(output_raster)
            # Check if the raster was saved successfully
            if os.path.exists(output_raster):
                saved_paths.append(output_raster)
                print(f"Clipped {key} successfully and saved to {output_raster}")
            else:
                print(f"Failed to save {key} to {output_raster}")
        except Exception as e:
            print(f"Error clipping {key}: {e}")
    return saved_paths

# Read the feature class and count the number of rows
with arcpy.da.SearchCursor(huc_path, ["SHAPE@", huc_col, id_col]) as cursor:
    rows = [row for row in cursor]

# Check the number of rows in the feature class
if len(rows) > 1:
    huc12s_dir = os.path.join(project_dir, "HUC12s")
    create_dir(huc12s_dir)
    
    all_saved_paths = []  # List to collect all saved paths

    for row in rows:
        # Extract HUC name and ID
        hucname = str(row[1]).title()
        hucid = str(row[2])
        # Define the output subdirectory name based on the HUC ID and HUC name
        subdir = hucid + '_' + hucname.replace(' ', '')
        output_folder = os.path.join(huc12s_dir, subdir)
        create_dir(output_folder)
        # Clip and save the rasters
        saved_paths = clip_and_save(row[0], output_folder)
        all_saved_paths.extend(saved_paths)  # Collect all paths

    # Define the mosaic output folder
    mosaic_dir = os.path.join(project_dir, "mosaic")
    create_dir(mosaic_dir)

    # Group files based on filename and extension
    file_groups = {}
    for file_path in all_saved_paths:
        file_name, file_ext = os.path.splitext(os.path.basename(file_path))
        if (file_name, file_ext) not in file_groups:
            file_groups[(file_name, file_ext)] = []
        file_groups[(file_name, file_ext)].append(file_path)

    # Mosaic each group separately
    for group, files in file_groups.items():
        mosaic_name = f"{group[0]}{group[1]}"
        print(f'Creating mosaic for {mosaic_name}')
        mosaic_path = os.path.join(mosaic_dir, mosaic_name)
        arcpy.MosaicToNewRaster_management(files, mosaic_dir, mosaic_name, pixel_type="32_BIT_FLOAT", number_of_bands=1)
    
else:
    basedir = os.path.join(project_dir, "basedir")
    create_dir(basedir)
    # Clip and save the rasters for the single HUC
    saved_paths = clip_and_save(rows[0][0], basedir)
    print(f"Saved paths: {saved_paths}")

# Clean up
arcpy.CheckInExtension("Spatial")


Clipped dem_ft successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\dem_ft.tif
Clipped wte_ft successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\wte_ft.tif
Clipped whc_ratio successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\whc_ratio.tif
Clipped impervious successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\impervious.tif
Clipped water successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\water.tif
Clipped rain_1d5yr_in successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\rain_1d5yr_in.tif
Clipped rain_1d10yr_in successfully and saved to X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\rain_1d10yr_in.tif
Clippe

'CheckedOut'


### Section 1.2 – ArcHydro Processing for Initial Stage and Time of Concentration

This section of the notebook uses ArcHydro tools to derive key hydrologic parameters such as **initial stage** and **time of concentration**. It also generates intermediate hydrologic datasets needed for flood modeling.

---

#### 🗂 1. Folder Structure and Input Preparation

Ensure your DEM files are properly organized in the following folder structure:

```
Project_Directory/
└── HUC12s/
    ├── {HUC_ID}_{HUC_Name}/
    │   └── dem_ft.tif
```

Use the same `project_dir` as defined in Section 1.1 for consistency.

---

#### 🏗 2. Create Geodatabase

The output geodatabase should be named `ArcHydro.gdb` and located in the root of your `project_dir`. If it doesn’t exist, the script will automatically create it.

Example:
```python
project_dir = r"X:\path\to\your\project"
geodatabase = os.path.join(project_dir, "ArcHydro.gdb")
```

---

#### ⚙️ 3. Processing Steps

The script uses the `glob` module to locate all DEM files under `HUC12s/` and applies ArcHydro preprocessing tools for each sub-watershed.

For each DEM:
- **Calculate Flow Direction**
- **Calculate Flow Accumulation**
- **Define Streams** (based on a threshold value)
- **Perform Stream Segmentation**
- **Calculate Stream Order**
- **Create Catchment Grid**
- **Convert Catchment Grid to Catchment Polygon**
- **Generate Drainage Lines**
- **Generate Drainage Points**

Each output is written to the `ArcHydro.gdb` geodatabase and is named using the HUC ID extracted from the subdirectory structure.

---

#### 📤 4. Output Management

All intermediate and final hydrologic layers are saved in:

```
Project_Directory/ArcHydro.gdb
```

Output files include:
- `flow_dir`
- `flow_acc`
- `streams`
- `stream_links`
- `catchment_grid`
- `catchment_polygons`
- `drainage_lines`
- `drainage_points`

Naming conventions are maintained using the corresponding HUC ID.

---

#### 🧹 5. Cleanup

To keep the environment clean, temporary layers and in-memory data used during processing are removed after each DEM is processed.

---

#### ✅ Final Notes

- Ensure all DEMs are projected and aligned before running the script.
- The script assumes that the ArcHydro toolbox is installed and licensed in your ArcGIS Pro environment.
- It is critical to maintain the folder structure and naming conventions to ensure accurate output referencing in downstream processing.



In [2]:
%%time
import os
import glob
import arcpy
from arcpy.sa import *

arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# Set the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'

# Path to the geodatabase
gdb_path = os.path.join(project_dir, 'ArcHydro.gdb')

# Create the geodatabase if it doesn't exist
if not arcpy.Exists(gdb_path):
    arcpy.CreateFileGDB_management(project_dir, 'ArcHydro.gdb')
    print(f"Geodatabase created at {gdb_path}")
else:
    print(f"Geodatabase already exists at {gdb_path}")

# Use the glob library to search for dem_ft.tif files in the specified pattern
dem_files = glob.glob(os.path.join(project_dir, 'HUC12s/**/dem_ft.tif'), recursive=True)

# Function to extract numeric part of HUC ID from directory name
def extract_hucid(directory):
    return ''.join(filter(str.isdigit, os.path.basename(directory)))

# Print the list of .tif files
for dem_file in dem_files:
    # Split the file path into directory and file name
    directory, file_name = os.path.split(dem_file)
    hucid = extract_hucid(directory)

    # Generate output names based on hucid
    fd_name = f"{hucid}_fd"
    fa_name = f"{hucid}_fa"
    sd_name = f"{hucid}_sd"
    sl_name = f"{hucid}_sl"
    so_name = f"{hucid}_so"
    cg_name = f"{hucid}_cg"
    cp_name = f"{hucid}_cp"
    dl_name = f"{hucid}_dl"
    dp_name = f"{hucid}_dp"

    try:
        # Step 1: Calculate Flow Direction
        flow_direction = FlowDirection(dem_file, "NORMAL")

        # Step 2: Calculate Flow Accumulation
        flow_accumulation = FlowAccumulation(flow_direction)

        # Step 3: Define Streams (Optional)
        # Adjust the threshold value as needed to define streams
        stream_definition = Con(flow_accumulation > 200, 5)

        # Step 4: Stream Segmentation
        stream_link = StreamLink(flow_direction, stream_definition)
        stream_order = StreamOrder(stream_link, flow_direction)

        # Step 5: Create Catchment Grid
        catchment_grid = Watershed(flow_direction, stream_link)

        # Step 6: Create Catchment Polygon
        catchment_polygon = arcpy.RasterToPolygon_conversion(catchment_grid, "in_memory/catchment_polygons", "NO_SIMPLIFY")

        # Step 7: Create Drainage Line
        drainage_line = StreamToFeature(stream_link, flow_direction, "in_memory/drainage_lines")

        # Step 8: Create Drainage Point
        drainage_point = arcpy.RasterToPoint_conversion(stream_link, "in_memory/drainage_points")

        """# Save the results to the geodatabase
        #flow_direction.save(os.path.join(gdb_path, fd_name))
        flow_accumulation.save(os.path.join(gdb_path, fa_name))
        stream_definition.save(os.path.join(gdb_path, sd_name))
        stream_link.save(os.path.join(gdb_path, sl_name))
        stream_order.save(os.path.join(gdb_path, so_name))
        catchment_grid.save(os.path.join(gdb_path, cg_name))
        """
        arcpy.CopyFeatures_management(catchment_polygon, os.path.join(gdb_path, cp_name))
        arcpy.CopyFeatures_management(drainage_line, os.path.join(gdb_path, dl_name))
        arcpy.CopyFeatures_management(drainage_point, os.path.join(gdb_path, dp_name))

        # Clean up temporary in-memory data
        arcpy.Delete_management("in_memory")
        print(f'Processed {dem_file}')

    except Exception as e:
        print(f"Error processing {dem_file}: {e}")

print("Hydrological analysis completed.")


Geodatabase created at X:\amandal2023$\Working\Homestead\Homestead_City\ArcHydro.gdb
Error processing X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\dem_ft.tif: Failed to execute. Parameters are not valid.
ERROR 000354: The name contains invalid characters
Failed to execute (CopyFeatures).

Error processing X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\dem_ft.tif: Failed to execute. Parameters are not valid.
ERROR 000354: The name contains invalid characters
Failed to execute (CopyFeatures).

Error processing X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\dem_ft.tif: Failed to execute. Parameters are not valid.
ERROR 000354: The name contains invalid characters
Failed to execute (CopyFeatures).

Hydrological analysis completed.
Wall time: 15min 47s



### Section 1.3 – Unsaturated Zone and Soil Storage Calculations

This section of the notebook calculates the **unsaturated zone depth** and **soil storage capacity** for each watershed using raster algebra.

---

#### ✅ 1. Set the Project Directory

Ensure the `project_dir` variable is correctly set to your working project path. This should point to the folder that contains your clipped input rasters.

Example:
```python
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'
```

---

#### ▶️ 2. Run the Script

Once the directory is set, running the script will:

- Loop through all `dem_ft.tif` files in the project.
- Locate and process corresponding input files for each DEM:
  - `wte_ft.tif` – Water Table Elevation
  - `whc_ratio.tif` – Water Holding Capacity
  - `impervious.tif` – Impervious Mask
  - `water.tif` – Water Mask

The outputs generated for each area include:
- `soilStorage.tif` – Soil storage capacity raster
- `unsaturated.tif` – Unsaturated zone depth raster

---

#### 🛠 3. Review and Adjust Variables if Necessary

The script identifies other input files by replacing parts of the `dem_ft.tif` filename. If your raster files are named differently, update the following replace statements accordingly:

```python
WTEft = dem_file.replace('dem_ft', 'wte_ft')
WHC = dem_file.replace('dem_ft', 'whc_ratio')
ImperviousBinary = dem_file.replace('dem_ft', 'impervious')
WaterBinary = dem_file.replace('dem_ft', 'water')
```

Ensure that all raster files are stored in the correct subdirectories and follow a consistent naming convention.

---

#### 📤 4. Output Files

The output rasters will be saved in the same directories as their corresponding input `dem_ft.tif` files:

- `soilStorage.tif`: Reflects how much water the soil can hold, based on land cover and storage parameters.
- `unsaturated.tif`: Represents the vertical depth from the ground surface to the water table.

Only positive values are retained in both outputs using conditional filtering (`VALUE > 0`).

---

#### 🗂 5. Mosaic Output (Optional)

After processing individual rasters, the script automatically:

- Collects all output paths
- Groups rasters based on filename
- Mosaics each group into a combined raster
- Saves mosaics in a folder named `mosaic` inside the `project_dir`

Make sure the `mosaic` directory exists or is created before saving. If needed, uncomment the following line to create the folder:

```python
#create_dir(mosaic_dir)
```

---

#### ✅ Final Note

This script requires the Spatial Analyst extension to be enabled in ArcGIS Pro. If you encounter any errors, check for:
- Missing input files
- Inconsistent raster naming
- Projection or cell size mismatches across rasters


In [3]:
%%time
import os
import glob
import arcpy
from arcpy.sa import *

arcpy.env.addOutputsToMap = False
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# Set the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'


all_saved_paths = []  # List to collect all saved paths
# Print the list of .tif files
for dem_file in dem_files:
    
    
    try:

        # Replace directory names to get corresponding file paths
        WTEft = dem_file.replace('dem_ft', 'wte_ft')
        WHC = dem_file.replace('dem_ft', 'whc_ratio')
        ImperviousBinary = dem_file.replace('DEMft', 'impervious')
        WaterBinary = dem_file.replace('dem_ft', 'water')

        # Define output file paths
        soilstorage = dem_file.replace('dem_ft', 'soilStorage')
        unsaturated = dem_file.replace('dem_ft', 'unsaturated')

        # Calculate soil storage
        soil_outras = RasterCalculator([dem_file, WTEft, WHC, ImperviousBinary, WaterBinary, 12],
                                       ["a", "b", "d", "e", "f", "g"], 
                                       "float((a-b)*d*e*f*g)")
        soil_outras.save(soilstorage)

        # Apply conditional to soil storage
        outdata = Con(soilstorage, soilstorage, 0, "VALUE >0")
        outdata.save(soilstorage)

        # Calculate unsaturated storage
        un_outras = RasterCalculator([dem_file, WTEft], ["a", "b"], "float(a-b)")
        un_outras.save(unsaturated)

        # Apply conditional to unsaturated storage
        outdata = Con(unsaturated, unsaturated, 0, "VALUE >0")
        outdata.save(unsaturated)
        print(f'Processed {dem_file}')
        all_saved_paths.extend([soilstorage,unsaturated])  # Collect all paths

    except Exception as e:
        print(f"Error processing {dem_file}: {e}")
# Define the mosaic output folder
mosaic_dir = os.path.join(project_dir, "mosaic")
#create_dir(mosaic_dir)

# Group files based on filename and extension
file_groups = {}
for file_path in all_saved_paths:
    file_name, file_ext = os.path.splitext(os.path.basename(file_path))
    if (file_name, file_ext) not in file_groups:
        file_groups[(file_name, file_ext)] = []
    file_groups[(file_name, file_ext)].append(file_path)

# Mosaic each group separately
for group, files in file_groups.items():
    mosaic_name = f"{group[0]}{group[1]}"
    print(f'Creating mosaic for {mosaic_name}')
    mosaic_path = os.path.join(mosaic_dir, mosaic_name)
    arcpy.MosaicToNewRaster_management(files, mosaic_dir, mosaic_name, pixel_type="32_BIT_FLOAT", number_of_bands=1)

print("Processing completed.")


Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061605_MoweryCanal\dem_ft.tif
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061606_CanalS177\dem_ft.tif
Processed X:\amandal2023$\Working\Homestead\Homestead_City\HUC12s\030902061607_ModelLandCanal\dem_ft.tif
Creating mosaic for soilStorage.tif
Creating mosaic for unsaturated.tif
Processing completed.
Wall time: 10min 7s



### Section 1.4 – Summarizing Inputs for Hydrologic Modeling

This section wraps up the input preparation process by compiling the necessary parameters into a single Excel table. These inputs are used to simulate flood scenarios under various design storm conditions using the Cascade 2001 hydrologic model.

---

#### 📄 Purpose

The goal of this step is to generate an Excel file that summarizes:
- Topographic characteristics of each sub-watershed
- Design rainfall inputs
- Soil storage capacity under various sea level rise or tidal scenarios

This Excel sheet serves as a key input to drive flood simulations using the Cascade model.

---

#### 📁 Input Source

This script pulls from:
- Digital Elevation Models (DEM) for max/min elevations
- Rainfall raster datasets averaged over each HUC
- Previously calculated soil storage rasters (one per scenario)

The storm scenarios and number of SSC columns depend on the parent folder naming or values specified in the first section of the script.

---

#### 🧾 Output Excel Table Format

The generated Excel file contains the following column headers:

| Column Name | Description |
|-------------|-------------|
| `HUC`       | Hydrologic Unit Code identifier for each sub-watershed |
| `MaxElevation` | Maximum elevation value from the DEM (ft) |
| `MinElevation` | Minimum elevation value from the DEM (ft) |
| `r1d5yr`    | Mean rainfall for 1-day, 5-year event (inches) |
| `r1d10yr`   | Mean rainfall for 1-day, 10-year event (inches) |
| `r1d100yr`  | Mean rainfall for 1-day, 100-year event (inches) |
| `r3d25yr`   | Mean rainfall for 3-day, 25-year event (inches) |
| `SCC0ft`    | Soil Storage Capacity under 0 ft sea level rise (inches) |
| `SCC1ft`    | Soil Storage Capacity under 1 ft sea level rise (inches) |
| `SCC2ft`    | Soil Storage Capacity under 2 ft sea level rise (inches) |
| `SCC3ft`    | Soil Storage Capacity under 3 ft sea level rise (inches) |
| `SCC4ft`    | Soil Storage Capacity under 4 ft sea level rise (inches) |
| `SCC5ft`    | Soil Storage Capacity under 5 ft sea level rise (inches) |

---

#### 📤 Output Location

The resulting Excel file is saved to a designated folder inside your `project_dir`, typically under:

```
project_dir/Cascade/Inputs/
```

The filename is auto-generated based on the base project folder name.

---

#### ✅ Final Notes

- Ensure all raster layers have been processed before running this step.
- Column names and scenario labels should be consistent with the earlier sections.
- The Excel sheet will be used in the next phase for headwater height calculations and flood simulation using the Cascade 2001 model.


In [4]:
%%time
import os
import glob
import pandas as pd
from arcpy.sa import *
arcpy.env.addOutputsToMap = False

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

# Enable overwriting of output files
arcpy.env.overwriteOutput = True

# Define the project directory
project_dir = r'X:\amandal2023$\Working\Homestead\Homestead_City'
gdb_path = os.path.join(project_dir, 'ArcHydro.gdb')

def stat(ras):
    minv = arcpy.GetRasterProperties_management(ras, "MINIMUM")
    maxv = arcpy.GetRasterProperties_management(ras, "MAXIMUM")
    meanv = arcpy.GetRasterProperties_management(ras, "MEAN")
    return [float(minv.getOutput(0)), float(maxv.getOutput(0)), float(meanv.getOutput(0))]

# Use the glob library to search for .tif files in subdirectories named "dem_ft.tif"
dem_files = glob.glob(os.path.join(project_dir, 'HUC12s/**/dem_ft.tif'), recursive=True)

# Define the output directory for the Cascade input file
cascade_output_dir = os.path.join(project_dir, 'Cascade', 'Inputs', 'table')
os.makedirs(cascade_output_dir, exist_ok=True)

# Use the base directory name for the Cascade input file name
base_dir_name = os.path.basename(project_dir)
cascadeinputs = os.path.join(cascade_output_dir, f'{base_dir_name}_Cascade.xlsx')

# Define the temp directory for scc outputs
temp_scc_dir = os.path.join(project_dir, 'Temp', 'scc')
os.makedirs(temp_scc_dir, exist_ok=True)

df_list = []

print(f'Processing directory: {base_dir_name}')
for dem_file in dem_files:
    dirs, filename = os.path.split(dem_file)
    HUC = dirs.split(os.sep)[-1].split('_')[0]  # Extract HUC ID from directory name
    
    dem = stat(dem_file)
    WTEft = dem_file.replace('dem_ft.tif', 'wte_ft.tif')
    WHC = dem_file.replace('dem_ft.tif', 'whc_ratio.tif')
    ImperviousBinary = dem_file.replace('dem_ft.tif', 'impervious.tif')
    WaterBinary = dem_file.replace('dem_ft.tif', 'water.tif')
    
    r1d5yr = stat(dem_file.replace('dem_ft.tif', 'rain_1d5yr_in.tif'))[2]
    r1d10yr = stat(dem_file.replace('dem_ft.tif', 'rain_1d10yr_in.tif'))[2]
    r1d100yr = stat(dem_file.replace('dem_ft.tif', 'rain_1d100yr_in.tif'))[2]
    r3d25yr = stat(dem_file.replace('dem_ft.tif', 'rain_3d25yr_in.tif'))[2]
    
    scclist = []
    for i in range(0, 6):
        output_raster = os.path.join(temp_scc_dir, f'{HUC}_{i}.tif')
        outras = RasterCalculator([dem_file, WTEft, i, WHC, ImperviousBinary, WaterBinary, 12],
                                  ["a", "b", "c", "d", "e", "f", "g"], 
                                  "float((a-(b+c))*d*e*f*g)")
        outras.save(output_raster)
        outdata = Con(output_raster, output_raster, 0, "VALUE >0")
        outdata.save(output_raster)
        scc = stat(output_raster)[2]
        scclist.append(scc)
    
    # Append the row to the list of dictionaries
    df_list.append({'HUC': HUC, 'MaxElevation': dem[1], 'MinElevation': dem[0], 'r1d5yr': r1d5yr,
                    'r1d10yr': r1d10yr, 'r1d100yr': r1d100yr, 'r3d25yr': r3d25yr,
                    'SCC0ft': scclist[0], 'SCC1ft': scclist[1],
                    'SCC2ft': scclist[2], 'SCC3ft': scclist[3], 'SCC4ft': scclist[4], 'SCC5ft': scclist[5]})

# Convert the list of dictionaries to a DataFrame and save to Excel
df = pd.DataFrame(df_list)
df.to_excel(cascadeinputs, index=False)
print(f'Completed processing for directory: {base_dir_name}')

print("Processing completed.")


Processing directory: Homestead_City
Completed processing for directory: Homestead_City
Processing completed.
Wall time: 28min 18s
